In [1]:
import sys
sys.path.append('..')
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('../data/raw/Suicide_Detection.csv', usecols=['text','class'])
df['class'] = df['class'].map({'suicide': 1, 'non-suicide':0})

In [3]:
length = [len(document.split()) for document in df['text']]
exclamation = [len([char for char in document if char == '!']) for document in df['text']]

In [4]:
df = pd.read_csv('../data/prepared/prepared.csv', index_col=0)
df['corpus'] = df['corpus'].apply(lambda x: x[1:-1].replace("'", "").split(', '))

In [5]:
def get_sentiment(text):
        sentiment = sid.polarity_scores(" ".join(text))
        help_words = [
        'help','suicide','plz', 'cyanide','ibuprofen','charcoal',
        'euthanasia','survivor','please','unimportant', 'insulin',
        'support', 'urgent', 'emergency']
        
        if any(word in text for word in help_words): 
            sentiment['compound'] *= 0.5
        return [value for value in sentiment.values()]
        
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
sentiment = df['corpus'].apply(lambda x: get_sentiment(x))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
vectorizer = TfidfVectorizer(min_df=4, lowercase=False)
X = vectorizer.fit_transform(df['corpus'].apply(lambda x: ' '.join(x)))
features = vectorizer.get_feature_names_out()

keyword = [features[document.toarray().argsort()[0][::-1][:3]] for document in X]

In [7]:
length = pd.DataFrame(length)
exclamation = pd.DataFrame(exclamation)

length = length[length.index.isin(df.index)]
exclamation = exclamation[exclamation.index.isin(df.index)]

In [9]:
df_feature = pd.concat(
    [length,
    exclamation,
    pd.DataFrame(sentiment.to_list(), index=df.index),
    pd.Series(keyword, index=df.index),
    df['class'],
    ],
    axis=1
)
df_feature.columns = ['length', 'exclamation', 'negative', 'neutral', 'positive', 'compound', 'keywords', 'class']
df_feature.index = df.index

In [10]:
df_feature

,length,exclamation,negative,neutral,positive,compound,keywords,class
0,143,0,0.398,0.441,0.161,-0.48700,"[wife, threatening, cheated]",1
1,27,0,0.225,0.529,0.245,0.09840,"[compliment, affected, irl]",0
2,26,0,0.259,0.433,0.308,0.20250,"[swear, annoying, hear]",0
3,9,0,0.441,0.294,0.265,-0.10115,"[helpjust, cry, hard]",1
4,438,0,0.361,0.513,0.126,-0.49795,"[anti, feeling, therapy]",1
...,...,...,...,...,...,...,...,...
232069,19,0,0.260,0.740,0.000,-0.27550,"[rock, anything, going]",0
232070,31,0,0.298,0.532,0.170,-0.34000,"[deprived, count, nightmare]",0
232071,14,0,0.000,0.848,0.152,0.36120,"[double_exclamation_mark, pee, salty]",0
232072,364,0,0.251,0.530,0.219,-0.44715,"[toughing, life, fucked]",1


In [11]:
df_feature.to_csv("../data/prepared/new_features.csv")